In [2]:
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

In [3]:
# hyperparameters

batch_size = 128
num_classes = 10
epochs = 10
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [5]:
# load cifar10 data

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
img_height, img_width, channel = X_train.shape[1], X_train.shape[2], X_train.shape[3]

# convert to one hot encoding
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [6]:
# Normalizing  (0 to 1)
# https://stackoverflow.com/questions/62783984/how-to-normalize-pixel-values-in-an-image-and-save-it
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255


In [7]:
X_train.shape

(50000, 32, 32, 3)

In [8]:
X_test.shape

(10000, 32, 32, 3)

In [9]:
# Dense Block

def denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False, padding='same')(relu)
        if dropout_rate > 0:
            Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
    return temp

# Transition Block

def transition(input, num_filter=12, dropout_rate=0.2):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression),(1,1),use_bias=False, padding='same')(relu)
    if dropout_rate > 0:
        Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

# Output layer

def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    flat = layers.Flatten()(AvgPooling)
    output = layers.Dense(num_classes, activation='softmax')(flat)
    return output

In [10]:
num_filter = 12
dropout_rate = 0
l = 12

input = layers.Input(shape=(img_height, img_width, channel))
First_Conv2D = layers.Conv2D(num_filter, (3,3), use_bias=False, padding='same')(input)

First_Block = denseblock(First_Conv2D, 6, dropout_rate)
First_Transition = transition(First_Block, 128, dropout_rate)

Second_Block = denseblock(First_Transition, 12, dropout_rate)
Second_Transition = transition(Second_Block, 128, dropout_rate)

Third_Block = denseblock(Second_Transition, 24, dropout_rate)
Third_Transition = transition(Third_Block,128, dropout_rate)

Last_Block = denseblock(Third_Transition,  16, dropout_rate)
output = output_layer(Last_Block)

In [11]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 12)   324         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 12)  48          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 32, 32, 12)   0           ['batch_normalization[0][0]']

                                                                                                  
 concatenate_7 (Concatenate)    (None, 32, 32, 36)   0           ['concatenate_6[0][0]',          
                                                                  'conv2d_8[0][0]']               
                                                                                                  
 batch_normalization_8 (BatchNo  (None, 32, 32, 36)  144         ['concatenate_7[0][0]']          
 rmalization)                                                                                     
                                                                                                  
 activation_8 (Activation)      (None, 32, 32, 36)   0           ['batch_normalization_8[0][0]']  
                                                                                                  
 conv2d_9 (Conv2D)              (None, 32, 32, 3)    972         ['activation_8[0][0]']           
          

                                                                                                  
 batch_normalization_16 (BatchN  (None, 16, 16, 82)  328         ['concatenate_14[0][0]']         
 ormalization)                                                                                    
                                                                                                  
 activation_16 (Activation)     (None, 16, 16, 82)   0           ['batch_normalization_16[0][0]'] 
                                                                                                  
 conv2d_17 (Conv2D)             (None, 16, 16, 6)    4428        ['activation_16[0][0]']          
                                                                                                  
 concatenate_15 (Concatenate)   (None, 16, 16, 88)   0           ['concatenate_14[0][0]',         
                                                                  'conv2d_17[0][0]']              
          

                                                                                                  
 activation_24 (Activation)     (None, 16, 16, 130)  0           ['batch_normalization_24[0][0]'] 
                                                                                                  
 conv2d_25 (Conv2D)             (None, 16, 16, 6)    7020        ['activation_24[0][0]']          
                                                                                                  
 concatenate_23 (Concatenate)   (None, 16, 16, 136)  0           ['concatenate_22[0][0]',         
                                                                  'conv2d_25[0][0]']              
                                                                                                  
 batch_normalization_25 (BatchN  (None, 16, 16, 136)  544        ['concatenate_23[0][0]']         
 ormalization)                                                                                    
          

 conv2d_33 (Conv2D)             (None, 8, 8, 12)     14688       ['activation_32[0][0]']          
                                                                                                  
 concatenate_30 (Concatenate)   (None, 8, 8, 148)    0           ['concatenate_29[0][0]',         
                                                                  'conv2d_33[0][0]']              
                                                                                                  
 batch_normalization_33 (BatchN  (None, 8, 8, 148)   592         ['concatenate_30[0][0]']         
 ormalization)                                                                                    
                                                                                                  
 activation_33 (Activation)     (None, 8, 8, 148)    0           ['batch_normalization_33[0][0]'] 
                                                                                                  
 conv2d_34

                                                                  'conv2d_41[0][0]']              
                                                                                                  
 batch_normalization_41 (BatchN  (None, 4, 4, 80)    320         ['concatenate_37[0][0]']         
 ormalization)                                                                                    
                                                                                                  
 activation_41 (Activation)     (None, 4, 4, 80)     0           ['batch_normalization_41[0][0]'] 
                                                                                                  
 conv2d_42 (Conv2D)             (None, 4, 4, 8)      5760        ['activation_41[0][0]']          
                                                                                                  
 concatenate_38 (Concatenate)   (None, 4, 4, 88)     0           ['concatenate_37[0][0]',         
          

 ormalization)                                                                                    
                                                                                                  
 activation_49 (Activation)     (None, 4, 4, 144)    0           ['batch_normalization_49[0][0]'] 
                                                                                                  
 conv2d_50 (Conv2D)             (None, 4, 4, 8)      10368       ['activation_49[0][0]']          
                                                                                                  
 concatenate_46 (Concatenate)   (None, 4, 4, 152)    0           ['concatenate_45[0][0]',         
                                                                  'conv2d_50[0][0]']              
                                                                                                  
 batch_normalization_50 (BatchN  (None, 4, 4, 152)   608         ['concatenate_46[0][0]']         
 ormalizat

In [12]:
print(len(model.layers))

211


In [13]:
# determine the loss function and the optimizer

model.compile(loss='categorical_crossentropy',
             optimizer=Adam(),
             metrics=['accuracy'])

In [14]:
filepath = "model1.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [15]:
# Image Augmentation
# https://medium.com/analytics-vidhya/image-augmentation-9b7be3972e27
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

datagen = ImageDataGenerator(width_shift_range=0.1,
                             height_shift_range=0.1,
                             rotation_range=10,
                             shear_range=0.1,
                             zoom_range=0.1,
                             horizontal_flip=True)

iterator_train = datagen.flow(X_train, y_train, batch_size=64)

In [14]:
model.fit(iterator_train, batch_size=256, epochs=10, callbacks=callbacks_list,verbose=1,validation_data=(X_test, y_test))

Epoch 1/10
782/782 [==============================] - ETA: 0s - loss: 1.3646 - accuracy: 0.5060
Epoch 1: loss improved from inf to 1.36457, saving model to model1.h5
782/782 [==============================] - 169s 198ms/step - loss: 1.3646 - accuracy: 0.5060 - val_loss: 1.2539 - val_accuracy: 0.5741
Epoch 2/10
782/782 [==============================] - ETA: 0s - loss: 0.9214 - accuracy: 0.6736
Epoch 2: loss improved from 1.36457 to 0.92142, saving model to model1.h5
782/782 [==============================] - 108s 138ms/step - loss: 0.9214 - accuracy: 0.6736 - val_loss: 0.8965 - val_accuracy: 0.6858
Epoch 3/10
782/782 [==============================] - ETA: 0s - loss: 0.7736 - accuracy: 0.7285
Epoch 3: loss improved from 0.92142 to 0.77363, saving model to model1.h5
782/782 [==============================] - 111s 142ms/step - loss: 0.7736 - accuracy: 0.7285 - val_loss: 1.0863 - val_accuracy: 0.6513
Epoch 4/10
782/782 [==============================] - ETA: 0s - loss: 0.6798 - accuracy: 

In [15]:
# Test the model
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 9s 26ms/step - loss: 0.5803 - accuracy: 0.8043
Test loss: 0.580264687538147
Test accuracy: 0.8043000102043152


- For 10 epochs, we got the Test Accuaracy of 80%

In [17]:
# loading the previously trained model and continuing to train to imporove test accuracy.

from tensorflow.keras.models import load_model
model = load_model('model1.h5')

# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# this time, we will train for 20 epochs

model.fit(iterator_train, batch_size=256, epochs=20, callbacks=callbacks_list,validation_data=(X_test, y_test), verbose=1)

Epoch 1/20
782/782 [==============================] - ETA: 0s - loss: 0.4404 - accuracy: 0.8460
Epoch 1: loss improved from 0.45501 to 0.44041, saving model to model1.h5
782/782 [==============================] - 115s 141ms/step - loss: 0.4404 - accuracy: 0.8460 - val_loss: 0.5821 - val_accuracy: 0.8056
Epoch 2/20
782/782 [==============================] - ETA: 0s - loss: 0.4146 - accuracy: 0.8542
Epoch 2: loss improved from 0.44041 to 0.41462, saving model to model1.h5
782/782 [==============================] - 107s 137ms/step - loss: 0.4146 - accuracy: 0.8542 - val_loss: 0.6521 - val_accuracy: 0.7968
Epoch 3/20
782/782 [==============================] - ETA: 0s - loss: 0.4012 - accuracy: 0.8611
Epoch 3: loss improved from 0.41462 to 0.40116, saving model to model1.h5
782/782 [==============================] - 110s 140ms/step - loss: 0.4012 - accuracy: 0.8611 - val_loss: 0.5612 - val_accuracy: 0.8192
Epoch 4/20
782/782 [==============================] - ETA: 0s - loss: 0.3803 - accura

In [18]:
# Test the model
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 8s 24ms/step - loss: 0.5419 - accuracy: 0.8373
Test loss: 0.5418946146965027
Test accuracy: 0.8373000025749207


- After retraining the same model for 20 epochs, we got the test accuracy of 83%

In [19]:
# loading the previously trained model and continuing to train to imporove test accuracy.

from tensorflow.keras.models import load_model
model = load_model('model1.h5')

# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# this time, we will train for 50 epochs

model.fit(iterator_train, batch_size=256, epochs=50, callbacks=callbacks_list,validation_data=(X_test, y_test), verbose=1)

Epoch 1/50
782/782 [==============================] - ETA: 0s - loss: 0.2435 - accuracy: 0.9134
Epoch 1: loss improved from 0.24512 to 0.24347, saving model to model1.h5
782/782 [==============================] - 111s 136ms/step - loss: 0.2435 - accuracy: 0.9134 - val_loss: 0.4518 - val_accuracy: 0.8609
Epoch 2/50
782/782 [==============================] - ETA: 0s - loss: 0.2348 - accuracy: 0.9168
Epoch 2: loss improved from 0.24347 to 0.23481, saving model to model1.h5
782/782 [==============================] - 108s 139ms/step - loss: 0.2348 - accuracy: 0.9168 - val_loss: 0.4549 - val_accuracy: 0.8574
Epoch 3/50
782/782 [==============================] - ETA: 0s - loss: 0.2338 - accuracy: 0.9167
Epoch 3: loss improved from 0.23481 to 0.23378, saving model to model1.h5
782/782 [==============================] - 118s 151ms/step - loss: 0.2338 - accuracy: 0.9167 - val_loss: 0.5030 - val_accuracy: 0.8517
Epoch 4/50
782/782 [==============================] - ETA: 0s - loss: 0.2286 - accura

Epoch 28/50
782/782 [==============================] - ETA: 0s - loss: 0.1587 - accuracy: 0.9436
Epoch 28: loss did not improve from 0.15702
782/782 [==============================] - 108s 138ms/step - loss: 0.1587 - accuracy: 0.9436 - val_loss: 0.4504 - val_accuracy: 0.8726
Epoch 29/50
782/782 [==============================] - ETA: 0s - loss: 0.1524 - accuracy: 0.9461
Epoch 29: loss improved from 0.15702 to 0.15243, saving model to model1.h5
782/782 [==============================] - 106s 135ms/step - loss: 0.1524 - accuracy: 0.9461 - val_loss: 0.4835 - val_accuracy: 0.8708
Epoch 30/50
782/782 [==============================] - ETA: 0s - loss: 0.1491 - accuracy: 0.9464
Epoch 30: loss improved from 0.15243 to 0.14910, saving model to model1.h5
782/782 [==============================] - 106s 135ms/step - loss: 0.1491 - accuracy: 0.9464 - val_loss: 0.4613 - val_accuracy: 0.8738
Epoch 31/50
782/782 [==============================] - ETA: 0s - loss: 0.1479 - accuracy: 0.9472
Epoch 31: los

In [20]:
# Test the model
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 8s 24ms/step - loss: 0.4363 - accuracy: 0.8860
Test loss: 0.43628519773483276
Test accuracy: 0.8859999775886536


- After retraining the same model for 50 more epochs, we got the test accuracy of 88%

In [21]:
# loading the previously trained model and continuing to train to imporove test accuracy.

from tensorflow.keras.models import load_model
model = load_model('model1.h5')

# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# this time, we will train for 20 epochs

model.fit(iterator_train, batch_size=256, epochs=20, callbacks=callbacks_list,validation_data=(X_test, y_test), verbose=1)

Epoch 1/20
782/782 [==============================] - ETA: 0s - loss: 0.1224 - accuracy: 0.9558
Epoch 1: loss did not improve from 0.12069
782/782 [==============================] - 110s 134ms/step - loss: 0.1224 - accuracy: 0.9558 - val_loss: 0.4663 - val_accuracy: 0.8812
Epoch 2/20
782/782 [==============================] - ETA: 0s - loss: 0.1217 - accuracy: 0.9559
Epoch 2: loss did not improve from 0.12069
782/782 [==============================] - 107s 136ms/step - loss: 0.1217 - accuracy: 0.9559 - val_loss: 0.4316 - val_accuracy: 0.8876
Epoch 3/20
782/782 [==============================] - ETA: 0s - loss: 0.1188 - accuracy: 0.9582
Epoch 3: loss improved from 0.12069 to 0.11885, saving model to model1.h5
782/782 [==============================] - 109s 139ms/step - loss: 0.1188 - accuracy: 0.9582 - val_loss: 0.4370 - val_accuracy: 0.8832
Epoch 4/20
782/782 [==============================] - ETA: 0s - loss: 0.1152 - accuracy: 0.9593
Epoch 4: loss improved from 0.11885 to 0.11516, sav

In [22]:
# Test the model
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 8s 24ms/step - loss: 0.4665 - accuracy: 0.8835
Test loss: 0.46650537848472595
Test accuracy: 0.8834999799728394


- After retraining the same model for 20 more epochs, we got the test accuracy of 88%
- We have trained the model for 100 epochs in total until now.

In [16]:
# loading the previously trained model and continuing to train to imporove test accuracy.

from tensorflow.keras.models import load_model
model = load_model('model1.h5')

# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# this time, we will train for 100 epochs

model.fit(iterator_train, batch_size=256, epochs=100, callbacks=callbacks_list,validation_data=(X_test, y_test), verbose=1)

Epoch 1/100
782/782 [==============================] - ETA: 0s - loss: 0.1039 - accuracy: 0.9632
Epoch 1: loss improved from inf to 0.10392, saving model to model1.h5
782/782 [==============================] - 119s 135ms/step - loss: 0.1039 - accuracy: 0.9632 - val_loss: 0.5125 - val_accuracy: 0.8803
Epoch 2/100
782/782 [==============================] - ETA: 0s - loss: 0.1031 - accuracy: 0.9635
Epoch 2: loss improved from 0.10392 to 0.10308, saving model to model1.h5
782/782 [==============================] - 102s 131ms/step - loss: 0.1031 - accuracy: 0.9635 - val_loss: 0.4477 - val_accuracy: 0.8852
Epoch 3/100
782/782 [==============================] - ETA: 0s - loss: 0.1001 - accuracy: 0.9636
Epoch 3: loss improved from 0.10308 to 0.10007, saving model to model1.h5
782/782 [==============================] - 102s 131ms/step - loss: 0.1001 - accuracy: 0.9636 - val_loss: 0.5503 - val_accuracy: 0.8661
Epoch 4/100
782/782 [==============================] - ETA: 0s - loss: 0.1024 - accura

782/782 [==============================] - ETA: 0s - loss: 0.0831 - accuracy: 0.9707
Epoch 29: loss improved from 0.08500 to 0.08307, saving model to model1.h5
782/782 [==============================] - 105s 134ms/step - loss: 0.0831 - accuracy: 0.9707 - val_loss: 0.6405 - val_accuracy: 0.8550
Epoch 30/100
782/782 [==============================] - ETA: 0s - loss: 0.0871 - accuracy: 0.9696
Epoch 30: loss did not improve from 0.08307
782/782 [==============================] - 104s 133ms/step - loss: 0.0871 - accuracy: 0.9696 - val_loss: 0.4459 - val_accuracy: 0.8901
Epoch 31/100
782/782 [==============================] - ETA: 0s - loss: 0.0837 - accuracy: 0.9700
Epoch 31: loss did not improve from 0.08307
782/782 [==============================] - 104s 133ms/step - loss: 0.0837 - accuracy: 0.9700 - val_loss: 0.4841 - val_accuracy: 0.8831
Epoch 32/100
782/782 [==============================] - ETA: 0s - loss: 0.0829 - accuracy: 0.9705
Epoch 32: loss improved from 0.08307 to 0.08291, savi

782/782 [==============================] - ETA: 0s - loss: 0.0698 - accuracy: 0.9757
Epoch 57: loss improved from 0.07041 to 0.06984, saving model to model1.h5
782/782 [==============================] - 103s 132ms/step - loss: 0.0698 - accuracy: 0.9757 - val_loss: 0.4976 - val_accuracy: 0.8899
Epoch 58/100
782/782 [==============================] - ETA: 0s - loss: 0.0738 - accuracy: 0.9743
Epoch 58: loss did not improve from 0.06984
782/782 [==============================] - 102s 131ms/step - loss: 0.0738 - accuracy: 0.9743 - val_loss: 0.6089 - val_accuracy: 0.8701
Epoch 59/100
782/782 [==============================] - ETA: 0s - loss: 0.0739 - accuracy: 0.9738
Epoch 59: loss did not improve from 0.06984
782/782 [==============================] - 108s 138ms/step - loss: 0.0739 - accuracy: 0.9738 - val_loss: 0.5089 - val_accuracy: 0.8822
Epoch 60/100
782/782 [==============================] - ETA: 0s - loss: 0.0718 - accuracy: 0.9748
Epoch 60: loss did not improve from 0.06984
782/782 [

Epoch 86/100
782/782 [==============================] - ETA: 0s - loss: 0.0669 - accuracy: 0.9763
Epoch 86: loss did not improve from 0.06093
782/782 [==============================] - 108s 138ms/step - loss: 0.0669 - accuracy: 0.9763 - val_loss: 0.5170 - val_accuracy: 0.8870
Epoch 87/100
782/782 [==============================] - ETA: 0s - loss: 0.0650 - accuracy: 0.9771
Epoch 87: loss did not improve from 0.06093
782/782 [==============================] - 108s 138ms/step - loss: 0.0650 - accuracy: 0.9771 - val_loss: 0.5510 - val_accuracy: 0.8821
Epoch 88/100
782/782 [==============================] - ETA: 0s - loss: 0.0619 - accuracy: 0.9786
Epoch 88: loss did not improve from 0.06093
782/782 [==============================] - 107s 136ms/step - loss: 0.0619 - accuracy: 0.9786 - val_loss: 0.5544 - val_accuracy: 0.8840
Epoch 89/100
782/782 [==============================] - ETA: 0s - loss: 0.0681 - accuracy: 0.9759
Epoch 89: loss did not improve from 0.06093
782/782 [==================

In [17]:
# Test the model
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 8s 24ms/step - loss: 0.5452 - accuracy: 0.8847
Test loss: 0.545221209526062
Test accuracy: 0.8847000002861023


- we have trained the model for 200 epochs in total
- we have got the test accuracy of 88%